In [ ]:
# default_exp capture_camera
# default_cls_lvl 2 

In [ ]:
#hide
from nbdev.showdoc import *

# Capture Camera
This script captures the camera pictures and saves it in folder. We will use it especially for the training phase. To make the module callable we use the very handy [fastscript library](https://github.com/fastai/fastscript).

In [ ]:
#export
import os
import argparse
from datetime import datetime
import time
import logging
import numpy as np
import cv2
import sys
from pathlib import Path
from logging.handlers import RotatingFileHandler
from dotenv import load_dotenv
from fastscript import *

In [ ]:
#export
def create_logger(log_file=None):
    """
    Initialize global logger and return it.

    :param log_file: log to this file, or to standard output if None
    :return: created logger
    """
    formatter = logging.Formatter(
        fmt='%(asctime)s %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')
    log = logging.getLogger()
    log.setLevel(logging.INFO)
    if log_file is not None:
        os.makedirs(os.path.dirname(log_file), exist_ok=True)
        handler = RotatingFileHandler(
            log_file,
            maxBytes=1024*1024*30,
            backupCount=3)
        handler.setFormatter(formatter)
        log.addHandler(handler)
        handler.setLevel(logging.DEBUG)

    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    handler.setLevel(logging.INFO)
    log.addHandler(handler)
    return log

In [ ]:
#export
def create_folder(path: Path, foldername):
    """ creates a folder if it does not exist already """
    (path / foldername).mkdir(parents=False, exist_ok=True)
    return path / foldername

In [ ]:
#export
@call_parse
def capture(
    path_save_files:Param('Path to where the pics will be stored.', str)='/mnt/Data/to_label',
    time_start:Param('time when recording starts', str)='8:00',
    time_stop:Param('time when recording stops', str)='17:00',
    file_log:Param('Path to where the log files will be stored', str)='/mnt/Data/capture_log',
    path_env_file:Param('Path where we can find the .env file', str)='..',
    sleep_time:Param('seconds to sleep between captures', float)=0.25):

    
    env_path = Path(path_env_file) / '.env'
    load_dotenv(dotenv_path=env_path)
    CAP_URL = os.getenv("CAP_URL")
    
    logger = create_logger(file_log)

    path = Path(path_save_files)
    
    time_start = int(
        datetime.strptime(time_start,
                          '%H:%M').strftime('%H%M'))

    time_stop = int(
        datetime.strptime(time_stop,
                          '%H:%M').strftime('%H%M'))

    current_date = datetime.today().date()
    prev_date = datetime.min.date()
    
    while True:
        time.sleep(10)
        logger.info('outer while loop')

        try:
            cap = cv2.VideoCapture(CAP_URL)
            cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
            while cap.isOpened():
                # read image
                ret, pic = cap.read()
                
                if ret is False:
                    logger.warning('pic not readable')
                    break

                now = datetime.now()
                current_date = datetime.today().date()                
                time_current = int(now.strftime('%H%M'))
                
                if time_start > time_current:
                    continue

                if time_stop < time_current:
                    continue

                if prev_date != current_date:
                    fld_save_to = create_folder(path, current_date.strftime('%Y%m%d'))
                    logger.info(f"msg=update folder to {current_date}")
                
                now_str = now.strftime('%Y%m%d%H%M%S_%f')
                cv2.imwrite(str(fld_save_to / (now_str + '.jpg')), pic)
                prev_date = datetime.today().date() #assign new date to compare to
                
                time.sleep(sleep_time)
                
        except Exception as e:
            logger.error(sleep_time)

        finally:
            cap.release()
            cv2.destroyAllWindows()

In [ ]:
now = datetime.now()
now

datetime.datetime(2020, 3, 16, 5, 51, 21, 33726)

In [ ]:
from nbdev.export import *
notebook2script('02_capture_camera.ipynb')


Converted 02_capture_camera.ipynb.


In [ ]:
#!python PooDetector/capture_camera.py

2020-03-09 07:54:08 outer while loop
^C
Traceback (most recent call last):
  File "PooDetector/capture_camera.py", line 62, in <module>
    sleep_time:Param('seconds to sleep between captures', float)=0.25):
  File "/home/wilhelm/miniconda3/envs/PooDetector/lib/python3.7/site-packages/fastscript/fastscript.py", line 43, in call_parse
    if mod.__name__=="__main__": return _f()
  File "/home/wilhelm/miniconda3/envs/PooDetector/lib/python3.7/site-packages/fastscript/fastscript.py", line 42, in _f
    func(**args.__dict__)
  File "PooDetector/capture_camera.py", line 93, in capture
    ret, pic = cap.read()
KeyboardInterrupt
